# lasagne
* lasagne is a library for neural network building and training
* it's a low-level library with almost seamless integration with theano

For a demo we shall solve the same digit recognition problem, but at a different scale
* images are now 28x28
* 10 different digits
* 50k samples

In [1]:
import numpy as np
import theano
import theano.tensor as T
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from mnist import load_dataset
X_train,y_train,X_val,y_val,X_test,y_test = load_dataset()

print(X_train.shape,y_train.shape)

(50000, 1, 28, 28) (50000,)


In [3]:
input_X = T.tensor4("X")

#input dimention (None means "Arbitrary" and only works at  the first axes [samples])
input_shape = [None,1,28,28]

target_y = T.vector("target Y integer",dtype='int32')

Defining network architecture

In [4]:
import lasagne
from lasagne.layers import *


import lasagne
from lasagne.layers import *

l1 = InputLayer(shape = input_shape,input_var=input_X)

l2 = Conv2DLayer(l1, 16, 5)

l3 = Pool2DLayer(l2, 2)

l4 = Conv2DLayer(l3, 32, 3)

l5 = Conv2DLayer(l4, 32, 3)

l6 = DenseLayer(l5,num_units=100,nonlinearity = lasagne.nonlinearities.elu)


l7 = DenseLayer(l6,num_units=150,nonlinearity = lasagne.nonlinearities.tanh)


l_out = DenseLayer(l7,num_units = 10,nonlinearity=lasagne.nonlinearities.softmax)

In [5]:
#network prediction (theano-transformation)
y_predicted = lasagne.layers.get_output(l_out)

In [6]:
#all network weights (shared variables)
all_weights = lasagne.layers.get_all_params(l_out)
print (all_weights)

[W, b, W, b, W, b, W, b, W, b, W, b]


### Than you could simply
* define loss function manually
* compute error gradient over all weights
* define updates
* But that's a whole lot of work and life's short
  * not to mention life's too short to wait for SGD to converge

Instead, we shall use Lasagne builtins

In [7]:
#Mean categorical crossentropy as a loss function - similar to logistic loss but for multiclass targets
loss = lasagne.objectives.categorical_crossentropy(y_predicted,target_y).mean()

#prediction accuracy
accuracy = lasagne.objectives.categorical_accuracy(y_predicted,target_y).mean()

#This function computes gradient AND composes weight updates just like you did earlier
updates_sgd = lasagne.updates.adamax(loss, all_weights,learning_rate=0.01)

In [8]:
#function that computes loss and updates weights
inp = [input_X,target_y]
train_fun = theano.function(inp ,[loss,accuracy],updates= updates_sgd)

#function that just computes accuracy
accuracy_fun = theano.function([input_X,target_y],accuracy)

### That's all, now let's train it!
* We got a lot of data, so it's recommended that you use SGD
* So let's implement a function that splits the training sample into minibatches

In [9]:
# An auxilary function that returns mini-batches for neural network training

#Parameters
# inputs - a tensor of images with shape (many, 1, 28, 28), e.g. X_train
# outputs - a vector of answers for corresponding images e.g. Y_train
#batch_size - a single number - the intended size of each batches

def iterate_minibatches(inputs, targets, batchsize):
    assert len(inputs) == len(targets)
    indices = np.arange(len(inputs))
    np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        excerpt = indices[start_idx:start_idx + batchsize]
        yield inputs[excerpt], targets[excerpt]

# Training loop

In [10]:
import time

num_epochs = 15 #amount of passes through the data
batch_size = 10 #number of samples processed at each function call

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train,batch_size):
        inputs, targets = batch
        #print(inputs.shape, targets.shape)
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1
    
    if (time.time() - start_time)%100 < 1:
        print( time.time() - start_time, 's')
    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))

    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(
        train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

Epoch 1 of 15 took 187.103s
  training loss (in-iteration):		0.145372
  train accuracy:		95.63 %
  validation accuracy:		97.53 %
Epoch 2 of 15 took 189.808s
  training loss (in-iteration):		0.060686
  train accuracy:		98.23 %
  validation accuracy:		98.34 %
Epoch 3 of 15 took 199.225s
  training loss (in-iteration):		0.044184
  train accuracy:		98.73 %
  validation accuracy:		98.80 %
Epoch 4 of 15 took 201.557s
  training loss (in-iteration):		0.032823
  train accuracy:		99.07 %
  validation accuracy:		98.80 %
Epoch 5 of 15 took 201.101s
  training loss (in-iteration):		0.028490
  train accuracy:		99.14 %
  validation accuracy:		98.44 %
Epoch 6 of 15 took 203.043s
  training loss (in-iteration):		0.024998
  train accuracy:		99.25 %
  validation accuracy:		99.10 %
Epoch 7 of 15 took 201.940s
  training loss (in-iteration):		0.020328
  train accuracy:		99.42 %
  validation accuracy:		98.82 %
Epoch 8 of 15 took 203.763s
  training loss (in-iteration):		0.017458
  train accuracy:		99.46 %


In [11]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 99:
    print ("Achievement unlocked: 80lvl Warlock!")
else:
    print ("We need more magic!")

Final results:
  test accuracy:		99.06 %
Achievement unlocked: 80lvl Warlock!


# Now improve it!

* Moar layers!
* Moar units!
* Different nonlinearities!